In [1]:
%run ../lib/Matrix_Factorization_A1_P2.py

In [2]:
data = pd.read_csv('../data/ml-latest-small/ratings.csv')

In [3]:
test_data = data.sample(frac =.99)
train_data = data.drop(test_data.index)

In [4]:
U = len(data['userId'].unique())
I = len(data['movieId'].unique())

In [ ]:
result = gradesc(f = 10,U=U,I=I,lam = 0.1,lrate = 0.1,stopping_deriv = 0.04,data = data, train_data=train_data, test_data=test_data)